In [8]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
import geopandas as gpd
import cartopy.crs as ccrs
from scipy.spatial import cKDTree
from shapely.geometry import Point
import concurrent.futures
from tensorflow.keras.models import load_model

from tensorflow.keras import backend as K


# Custom Metrics : NRMSE
def nrmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred))) / (K.max(y_pred) - K.min(y_pred))


# Custom Metrics : RMSE
def root_mse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))


class Data:

    def __init__(self, path_to_file, name):
        self.path_to_file = path_to_file
        self.name = name

    @staticmethod
    def x_y_to_stacked_xy(x_array, y_array):
        stacked_xy = np.dstack((x_array, y_array))
        return (stacked_xy)

    @staticmethod
    def grid_to_flat(stacked_xy):
        x_y_flat = [tuple(i) for line in stacked_xy for i in line]
        return (x_y_flat)

    @property
    def length(self):
        return (self.shape[1])

    @property
    def height(self):
        return (self.shape[0])


class MNT(Data):

    def __init__(self, path_to_file, name):
        super().__init__(path_to_file, name)
        self.data_xr = xr.open_rasterio(path_to_file)
        self.data = self.data_xr.values[0, :, :]

    def find_nearest_MNT_index(self, x, y):
        xmin_MNT = np.min(self.data_xr.x.data)
        ymax_MNT = np.max(self.data_xr.y.data)

        index_x_MNT = (x - xmin_MNT) // self.resolution_x
        index_y_MNT = (ymax_MNT - y) // self.resolution_y
        return (index_x_MNT, index_y_MNT)

    @property
    def shape(self):
        return (self.data.shape)

    @property
    def resolution_x(self):
        return (self.data_xr.res[0])

    @property
    def resolution_y(self):
        return (self.data_xr.res[1])


class NWP(Data):

    def __init__(self, path_to_file, name, begin, end):
        super().__init__(path_to_file, name)
        self.data_xr = xr.open_dataset(path_to_file)
        self.begin = begin
        self.end = end

    @property
    def shape(self):
        return (self.data_xr['LON'].shape)

    def gps_to_l93(self):
        """Converts LAT/LON information from a NWP to L93"""

        # Initialization
        X_L93 = np.zeros(self.shape)
        Y_L93 = np.zeros(self.shape)

        # Load transformer
        gps_to_l93_func = pyproj.Transformer.from_crs(4326, 2154, always_xy=True)

        # Transform coordinates of each points
        for i in range(self.height):
            for j in range(self.length):
                projected_points = [point for point in
                                    gps_to_l93_func.itransform(
                                        [(self.data_xr['LON'][i, j], self.data_xr['LAT'][i, j])])]
                X_L93[i, j], Y_L93[i, j] = projected_points[0]

        # Create a new variable with new coordinates
        self.data_xr["X_L93"] = (("yy", "xx"), X_L93)
        self.data_xr["Y_L93"] = (("yy", "xx"), Y_L93)

    def select_timeframe(self, variables):
        self.data_xr = self.data_xr[variables].sel(time=slice(begin, end))


class Observation():

    def __init__(self, path_to_list_stations, path_to_time_series):
        self.path_to_list_stations = path_to_list_stations
        self.path_to_time_series = path_to_time_series
        self.stations = pd.read_csv(path_to_list_stations)
        self.time_series = pd.read_csv(path_to_time_series)

    def stations_to_gdf(self, from_epsg, x="LON", y="LAT"):
        """
        Input: Dataframe 1D
        Output: GeoDataFrame 1D
        """

        if from_epsg == "epsg:4326":
            crs = {"init": from_epsg}
        else:
            crs = from_epsg

        self.stations = gpd.GeoDataFrame(self.stations,
                                         geometry=gpd.points_from_xy(self.stations[x], self.stations[y]),
                                         crs=crs)

    def update_stations_with_KNN_from_NWP(self, number_neighbor, nwp):
        """Update a Observations.station (DataFrame) with index of NN in nwp

        ex: BDclim.update_stations_with_KNN_from_NWP(4, AROME) gives information about the 4 KNN at the each BDclim
        station from AROME
        """

        def K_N_N_point(point):
            distance, idx = tree.query(point, k=number_neighbor)
            return (distance, idx)

        # Reference stations
        list_coord_station = zip(self.stations['X'].values, self.stations['Y'].values)

        # Coordinates where to find neighbors
        stacked_xy = Data.x_y_to_stacked_xy(nwp.data_xr["X_L93"], nwp.data_xr["Y_L93"])
        grid_flat = Data.grid_to_flat(stacked_xy)
        tree = cKDTree(grid_flat)

        # Parallel computation of nearest neighbors
        with concurrent.futures.ThreadPoolExecutor() as executor:
            list_nearest = executor.map(K_N_N_point, list_coord_station)

        # Store results as array
        list_nearest = np.array([np.array(station) for station in list_nearest])
        list_index = [(x, y) for x in range(nwp.height) for y in range(nwp.length)]

        # Update DataFrame
        for neighbor in range(number_neighbor):
            self.stations[f'delta_x_{nwp.name}_NN_{neighbor}'] = list_nearest[:, 0, neighbor]
            self.stations[f'{nwp.name}_NN_{neighbor}'] = [Point(grid_flat[int(index)]) for index in
                                                          list_nearest[:, 1, neighbor]]
            self.stations[f'index_{nwp.name}_NN_{neighbor}'] = [list_index[int(index)] for index in
                                                                list_nearest[:, 1, neighbor]]

    def update_stations_with_KNN_from_MNT(self, mnt):
        index_x_MNT, index_y_MNT = mnt.find_nearest_MNT_index(self.stations["X"], self.stations["Y"])
        self.stations[f"index_X_NN_{mnt.name}"] = index_x_MNT
        self.stations[f"index_Y_NN_{mnt.name}"] = index_y_MNT

    def extract_MNT_around_station(self, station, mnt, nb_pixel_x, nb_pixel_y):
        condition = self.stations["name"] == station
        index_x = int(self.stations[[f"index_X_NN_{mnt.name}"]][condition].values[0])
        index_y = int(self.stations[[f"index_Y_NN_{mnt.name}"]][condition].values[0])
        MNT_data = mnt.data[index_y - nb_pixel_y:index_y + nb_pixel_y, index_x - nb_pixel_x:index_x + nb_pixel_x]
        MNT_x = mnt.data_xr.x.data[index_x - nb_pixel_x:index_x + nb_pixel_x]
        MNT_y = mnt.data_xr.x.data[index_y - nb_pixel_y:index_y + nb_pixel_y]
        return (MNT_data, MNT_x, MNT_y)


class Processing:
    n_rows, n_col = 79, 69

    def __init__(self, obs, mnt, nwp, model_path):
        self.observation = obs
        self.mnt = mnt
        self.nwp = nwp
        self.model_path = model_path

    @staticmethod
    def rotate_topography(topography, wind_dir, clockwise=False):
        """Rotate a topography to a specified angle

        If wind_dir = 270° then angle = 270+90 % 360 = 360 % 360 = 0
        For wind coming from the West, there is no rotation
        """
        if not (clockwise):
            rotated_topography = rotate(topography, 90 + wind_dir, reshape=False, mode='constant', cval=np.nan)
        if clockwise:
            rotated_topography = rotate(topography, -90 - wind_dir, reshape=False, mode='constant', cval=np.nan)
        return (rotated_topography)

    def rotate_topo_for_all_station(self):
        """Rotate the topography at all stations for each 1 degree angle of wind direction"""

        def rotate_topo_for_all_degrees(self, station):
            dict_topo[station]["rotated_topo_HD"] = {}
            MNT_data, _, _ = observation.extract_MNT_around_station(self, station, mnt, 400, 400)
            for angle in range(360):
                tile = self.rotate_topography(MNT_data, angle)
                dict_topo[station]["rotated_topo_HD"][str(angle)] = []
                dict_topo[station]["rotated_topo_HD"][str(angle)].append(tile)
            return (dict_topo)

        dict_topo = {}
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(rotate_topo_for_all_degrees, self.observation.stations['name'].values)
        self.dict_rot_topo = dict_topo

    @staticmethod
    def load_rotated_topo(wind_dir, station_name):
        angle_int = np.int64(np.round(wind_dir) % 360)
        angle_str = [str(angle) for angle in angle_int]
        topo_HD = [self.dict_rot_topo[station_name]["rotated_topo_HD"][angle][0][200 - 39:200 + 40, 200 - 34:200 + 35]
                   for angle in angle_str]
        return (topo_HD)

    @staticmethod
    def normalize_topo(topo_HD, mean, std):
        return ((np.array(topo_HD) - mean) / std)

    def select_nwp_pixel(self, station_name):
        nwp_name = self.nwp.name
        stations = self.observation.stations
        (x_idx_nwp, y_idx_nwp) = stations[f"index_{nwp_name}_NN_0"][stations["name"] == station_name].values[0]
        nwp_instance = self.nwp.data_xr
        wind_dir = nwp_instance.Wind_DIR[:, x_idx_nwp, y_idx_nwp]
        wind_speed = nwp_instance.Wind[:, x_idx_nwp, y_idx_nwp]
        return (wind_dir, wind_speed)

    def load_model(self, dependencies=False):
        # todo load dependencies
        if dependencies:

            def root_mse(y_true, y_pred):
                return K.sqrt(K.mean(K.square(y_true - y_pred)))

            dependencies = {'root_mse': root_mse}
            model = load_model(self.model_path + "/fold_0.h5", custom_objects=dependencies)
        else:
            model = load_model(self.model_path + "/fold_0.h5")
        self.model = model

    def load_norm_prm(self):
        # todo load dependencies
        dict_norm = pd.read_csv(self.model_path + "/dict_norm.csv")
        mean = dict_norm["0"].iloc[0]
        std = dict_norm["0"].iloc[1]
        return (mean, std)

    def predict_UV_with_CNN(self, stations_name, fast):

        # Select timeframe
        self.nwp.select_timeframe(['Wind', 'Wind_DIR'])

        # Station names
        if stations_name == 'all':
            stations_name = self.observation.stations["name"].values

        # If fast: pre-rotate all topo
        if fast:
            self.rotate_topo_for_all_station()

        for idx_station, single_station in enumerate(stations_name):

            # Select nwp pixel
            wind_dir, wind_speed = self.select_nwp_pixel(single_station)
            print('Selected pixel')
            if fast:
                print('Fast')
                rotated_topo = load_rotated_topo(wind_dir, single_station)
            else:
                print('Not fast')
                for time_step, angle in enumerate(wind_dir):
                    print('time_step, angle')
                    print(time_step, angle)
                    # Extract topography
                    topo_HD, _, _ = self.observation.extract_MNT_around_station(single_station,
                                                                                self.mnt,
                                                                                400,
                                                                                400)
                    print("Topography extracted")

                    # Rotate topography
                    rotated_topo_large = rotate_topography(topo_HD, angle)
                    rotated_topo = rotated_topo_large[200 - 39:200 + 40, 200 - 34:200 + 35]
                    print("Topography rotated")

                    # Reshape
                    rotated_topo = rotated_topo.reshape((1, n_rows, n_col, 1))
                    print("Topography reshaped")

                    # Concatenate
                    print("begin concatenation")
                    if time_step == 0:
                        print("Concat init begin")
                        all_time_steps = rotated_topo
                        print("Concat init done")
                    else:
                        print("Concat begin")
                        all_time_steps = np.concatenate((all_time_steps, rotated_topo), axis=0)
                        print("Concat done")

                rotated_topo = all_time_steps

            # Normalize
            mean, std = self.load_norm_prm()
            topo_norm = self.normalize_topo(rotated_topo, mean, std)

            # Reshape
            nb_sim = len(topo_norm)
            topo_reshaped = topo_norm.reshape((1, nb_sim, n_rows, n_col, 1))
            wind_speed = wind_speed.reshape((1, nb_sim))
            wind_dir = wind_dir.reshape((1, nb_sim))

            # Concatenate
            if idx_station == 0:
                topo_all = topo_reshaped
                wind_speed_all = wind_speed
                wind_dir_all = wind_dir
            if idx_station > 0:
                topo_all = np.concatenate((topo_all, topo_reshaped), axis=0)
                wind_speed_all = np.concatenate((wind_speed_all, wind_speed), axis=0)
                wind_dir_all = np.concatenate((wind_dir_all, wind_dir), axis=0)

        # Prediction
        """
        Warning: change dependencies here
        """
        self.load_model(dependencies=True)
        prediction = self.model.predict(topo_all)

        # Wind speed scaling
        for station in len(stations_name):
            for time_step in range(nb_sim):
                prediction = wind_speed_all[station, time_step] * prediction[station, time_step, :, :, :] / 3

        # Wind computations
        U_rot = prediction_i[:, :, :, :, 0]
        V_rot = prediction_i[:, :, :, :, 1]
        UV = np.sqrt(U_rot ** 2 + V_rot ** 2)
        alpha = np.arctan(U_rot / V_rot)
        for station in len(stations_name):
            for time_step in range(nb_sim):
                UV_DIR_rad = (np.pi / 180) * wind_dir_all[station, time_step] - alpha[station, time_step, :, :]
        U = -np.sin(UV_DIR_rad) * UV
        V = -np.cos(UV_DIR_rad) * UV

        # Rotate clockwise to put the wind value on the right topography pixel
        for idx_station in range(len(stations_name)):
            for time_step in range(nb_sim):
                U = rotate_topography(U[idx_station, time_step, :, :],
                                      wind_dir_all[idx_station, time_step],
                                      clockwise=True)
                V = rotate_topography(V[idx_station, time_step, :, :],
                                      wind_dir_all[idx_station, time_step],
                                      clockwise=True)




In [ ]:
data_path = "C:/Users/louis/git/wind_downscaling_CNN/Data/1_Raw/"
topo_path = data_path + "Topo/IGN_25m/ign_L93_25m_alpesIG.tif"
AROME_path = data_path + "AROME/FORCING_alp_2019060107_2019070106.nc"
BDclim_stations_path = data_path + "BDclim/precise_localisation/liste_postes_alps_l93.csv"
BDclim_data_path = data_path + "BDclim/extract_BDClim_et_sta_alp_20171101_20190501.csv"
experience_path = "C:/Users/louis/git/wind_downscaling_CNN/Models/ARPS/"
model_experience = "date_16_02_name_simu_FINAL_1_0_model_UNet/"
model_path = experience_path + model_experience

day_1 = 1
day_2 = 31
month = 1
year = 2019
begin = str(year) + "-" + str(month) + "-" + str(day_1)
end = str(year) + "-" + str(month) + "-" + str(day_2)

# IGN
IGN = MNT(topo_path, "IGN")

# AROME
AROME = NWP(AROME_path, "AROME", begin, end)
AROME.gps_to_l93()

# BDclim
BDclim = Observation(BDclim_stations_path, BDclim_data_path)
# BDclim.stations_to_gdf(ccrs.epsg(2154), x="X", y="Y")
number_of_neighbors = 4
BDclim.update_stations_with_KNN_from_NWP(number_of_neighbors, AROME)
BDclim.update_stations_with_KNN_from_MNT(IGN)
# MNT_data, MNT_x, MNT_y = BDclim.extract_MNT_around_station('Col du Lac Blanc', IGN, 100, 100)

In [9]:
# Processing
p = Processing(BDclim, IGN, AROME, model_path)
p.predict_UV_with_CNN(['Col du Lac Blanc'], fast=False)

Selected pixel
Not fast


TypeError: object of type 'numpy.float64' has no len()

In [27]:
stations_name = ['Col du Lac Blanc']
fast = False

def predict_UV_with_CNN(self, stations_name, fast):

    # Select timeframe
    self.nwp.select_timeframe(['Wind', 'Wind_DIR'])

    # Station names
    if stations_name == 'all':
        stations_name = self.observation.stations["name"].values

    # If fast: pre-rotate all topo
    if fast:
        self.rotate_topo_for_all_station()

    for idx_station, single_station in enumerate(stations_name):

        # Select nwp pixel
        wind_dir, wind_speed = self.select_nwp_pixel(single_station)
        print('Selected pixel')
        if fast:
            print('Fast')
            rotated_topo = load_rotated_topo(wind_dir, single_station)
        if not(fast):
            print('Not fast')
            print(wind_dir)
            for time_step, angle in enumerate(wind_dir):
                print('time_step, angle')
                print(time_step, angle)
                # Extract topography
                topo_HD, _, _ = self.observation.extract_MNT_around_station(single_station,
                                                                            self.mnt,
                                                                            400,
                                                                            400)
                print("Topography extracted")

                # Rotate topography
                rotated_topo_large = rotate_topography(topo_HD, angle)
                rotated_topo = rotated_topo_large[200 - 39:200 + 40, 200 - 34:200 + 35]
                print("Topography rotated")

                # Reshape
                rotated_topo = rotated_topo.reshape((1, n_rows, n_col, 1))
                print("Topography reshaped")

                # Concatenate
                print("begin concatenation")
                if time_step == 0:
                    print("Concat init begin")
                    all_time_steps = rotated_topo
                    print("Concat init done")
                else:
                    print("Concat begin")
                    all_time_steps = np.concatenate((all_time_steps, rotated_topo), axis=0)
                    print("Concat done")

            rotated_topo = all_time_steps

        # Normalize
        mean, std = self.load_norm_prm()
        topo_norm = self.normalize_topo(rotated_topo, mean, std)

        # Reshape
        nb_sim = len(topo_norm)
        topo_reshaped = topo_norm.reshape((1, nb_sim, n_rows, n_col, 1))
        wind_speed = wind_speed.reshape((1, nb_sim))
        wind_dir = wind_dir.reshape((1, nb_sim))

        # Concatenate
        if idx_station == 0:
            topo_all = topo_reshaped
            wind_speed_all = wind_speed
            wind_dir_all = wind_dir
        if idx_station > 0:
            topo_all = np.concatenate((topo_all, topo_reshaped), axis=0)
            wind_speed_all = np.concatenate((wind_speed_all, wind_speed), axis=0)
            wind_dir_all = np.concatenate((wind_dir_all, wind_dir), axis=0)

    # Prediction
    """
    Warning: change dependencies here
    """
    self.load_model(dependencies=True)
    prediction = self.model.predict(topo_all)

    # Wind speed scaling
    for station in len(stations_name):
        for time_step in range(nb_sim):
            prediction = wind_speed_all[station, time_step] * prediction[station, time_step, :, :, :] / 3

    # Wind computations
    U_rot = prediction_i[:, :, :, :, 0]
    V_rot = prediction_i[:, :, :, :, 1]
    UV = np.sqrt(U_rot ** 2 + V_rot ** 2)
    alpha = np.arctan(U_rot / V_rot)
    for station in len(stations_name):
        for time_step in range(nb_sim):
            UV_DIR_rad = (np.pi / 180) * wind_dir_all[station, time_step] - alpha[station, time_step, :, :]
    U = -np.sin(UV_DIR_rad) * UV
    V = -np.cos(UV_DIR_rad) * UV

    # Rotate clockwise to put the wind value on the right topography pixel
    for idx_station in range(len(stations_name)):
        for time_step in range(nb_sim):
            U = rotate_topography(U[idx_station, time_step, :, :],
                                  wind_dir_all[idx_station, time_step],
                                  clockwise=True)
            V = rotate_topography(V[idx_station, time_step, :, :],
                                  wind_dir_all[idx_station, time_step],
                                  clockwise=True)

In [28]:
predict_UV_with_CNN(p, stations_name, fast)

Selected pixel
Not fast
<xarray.DataArray 'Wind_DIR' (time: 0)>
array([], dtype=float64)
Coordinates:
  * time     (time) datetime64[ns] 
Attributes:
    grid_mapping:  Projection_parameters


UnboundLocalError: local variable 'all_time_steps' referenced before assignment